In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error



In [2]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

In [3]:
# Split the Iris dataset
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris, y_iris, test_size=0.2, random_state=42
)

In [4]:
# 2. Custom Decision Tree Implementation
class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1 or depth == self.max_depth:
            return {"class": np.bincount(y).argmax()}

        best_info_gain = -1
        best_split = None
        
        # Find the best split
        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask

                left_y = y[left_mask]
                right_y = y[right_mask]

                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                info_gain = self._information_gain(y, left_y, right_y)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        "feature_idx": feature_idx,
                        "threshold": threshold,
                        "left_mask": left_mask,
                        "right_mask": right_mask,
                    }

        if not best_split:
            return {"class": np.bincount(y).argmax()}

        # Recursively build the tree
        left_tree = self._build_tree(X[best_split["left_mask"]], y[best_split["left_mask"]], depth + 1)
        right_tree = self._build_tree(X[best_split["right_mask"]], y[best_split["right_mask"]], depth + 1)

        return {
            "feature_idx": best_split["feature_idx"],
            "threshold": best_split["threshold"],
            "left_tree": left_tree,
            "right_tree": right_tree,
        }

    def _information_gain(self, parent, left, right):
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_avg = (
            (len(left) / len(parent)) * left_entropy + (len(right) / len(parent)) * right_entropy
        )
        return parent_entropy - weighted_avg

    def _entropy(self, y):
        class_probs = np.bincount(y) / len(y)
        return -np.sum(class_probs * np.log2(class_probs + 1e-9))

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _predict_single(self, x, tree):
        if "class" in tree:
            return tree["class"]

        feature_val = x[tree["feature_idx"]]
        if feature_val <= tree["threshold"]:
            return self._predict_single(x, tree["left_tree"])
        else:
            return self._predict_single(x, tree["right_tree"])


In [5]:
# Train and evaluate the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train_iris, y_train_iris)
y_pred_custom = custom_tree.predict(X_test_iris)
accuracy_custom = accuracy_score(y_test_iris, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 1.0000


In [6]:
# 3. Scikit-learn Decision Tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train_iris, y_train_iris)
y_pred_sklearn = sklearn_tree.predict(X_test_iris)
accuracy_sklearn = accuracy_score(y_test_iris, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


In [7]:
# 4. Ensemble Methods with the Wine Dataset
wine = load_wine()
X_wine = wine.data
y_wine = wine.target

# Split the Wine dataset
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(
    X_wine, y_wine, test_size=0.2, random_state=42
)

# Train and evaluate classifiers
rf_clf = RandomForestClassifier(random_state=42)
dt_clf = DecisionTreeClassifier(random_state=42)

rf_clf.fit(X_train_wine, y_train_wine)
dt_clf.fit(X_train_wine, y_train_wine)

rf_f1 = f1_score(y_test_wine, rf_clf.predict(X_test_wine), average="weighted")
dt_f1 = f1_score(y_test_wine, dt_clf.predict(X_test_wine), average="weighted")

print(f"Random Forest F1 Score: {rf_f1:.4f}")
print(f"Decision Tree F1 Score: {dt_f1:.4f}")


Random Forest F1 Score: 1.0000
Decision Tree F1 Score: 0.9440


In [8]:
# 5. Hyperparameter Tuning
# GridSearchCV for RandomForestClassifier
param_grid = {
    "n_estimators": [50, 100, 150],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, scoring="f1_weighted")
grid_search.fit(X_train_wine, y_train_wine)
print(f"Best Parameters from GridSearchCV: {grid_search.best_params_}")

# RandomizedSearchCV for RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=42)
param_dist = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
}
random_search = RandomizedSearchCV(rf_reg, param_dist, n_iter=10, scoring="neg_mean_squared_error", random_state=42)
random_search.fit(X_train_wine, y_train_wine)
print(f"Best Parameters from RandomizedSearchCV: {random_search.best_params_}")


Best Parameters from GridSearchCV: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best Parameters from RandomizedSearchCV: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 10}
